In [1]:
import examples, core 
import os
import yaml
import dspy
import metrics

In [2]:
with open(os.path.join("..", "keys.yaml"), "r") as file:
    config = yaml.safe_load(file)
    openai_api_key = config["openai_api_key"]

llm = dspy.OpenAI(model='gpt-4o', api_key=openai_api_key, max_tokens=2000)

In [3]:
data = examples.load_examples("examples.json")

exp_metrics = metrics.Metrics(
            [
                metrics.accuracy,
                metrics.fluency,
                metrics.conciseness,
                metrics.context_awareness,
            ], verbose=1
        )

explingo = core.Explingo(llm=llm, context="The model predicts house prices", 
                         examples=data, metric=exp_metrics)

In [4]:
test_explanations = [d.explanation for d in data]
test_explanation_format = data[0].explanation_format

gold_standards = [d for d in data if hasattr(d, "narrative")]
max_optimal_length = max([len(d.narrative) for d in gold_standards])
max_optimal_length

474

In [5]:
VERIFY = True

if VERIFY:
    ver_metrics = metrics.Metrics(
            [
                metrics.accuracy,
                metrics.fluency,
                metrics.conciseness,
            ], 
        verbose=0, 
        metric_kwargs={"conciseness": {"max_optimal_length": max_optimal_length},
                       "fluency": {"good_narrative": gold_standards[0].narrative, "bad_narrative": gold_standards[1].narrative}}
    )
    print(ver_metrics(gold_standards[0], gold_standards[0]))

(6.0, accuracy       2.0
fluency        2.0
conciseness    2.0
dtype: float64)


# BASIC PROMPTING

TODO: update Explingo's prompting method to take in multiple different prompt options. 
DSPy does not support this by default, but we are using DSPy's evaluations

In [ ]:
explingo.run_experiment(data, prompt_type="basic", max_iters=5)

Narrative: The model predicts the house price based on several key features. The above ground living area of 1256 square feet decreases the predicted price by $12,527.46. The overall material and finish rating of the house, which is 5, reduces the price by $10,743.76. The absence of a second floor contributes to a decrease of $10,142.29. The house's location within the Edwards neighborhood in Ames city limits lowers the price by $9,913.81. However, the presence of a wood deck area of 736 square feet increases the predicted price by $9,846.38.
Total Score: 8.0
accuracy: 2.0, fluency: 2.0, conciseness: 2.0, context_awareness: 2.0, 
--
Narrative: The model predicts the house price based on several key features. The second floor square feet, which is 854.00, contributes significantly to increasing the price by 12757.84 units. The original construction date of the house, which is 2003.00, also adds a substantial amount of 9115.72 units to the price. On the other hand, the total square feet 

In [ ]:
explingo.run_experiment(data, prompt_type="few-shot", max_iters=5)

In [ ]:
explingo.run_experiment(data, prompt_type="bootstrap-few-shot", max_iters=5)